# Librerías

In [1]:
import configparser
import pymysql
import psycopg2

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

mysql_config = {
    'host': config['local_mysql']['HOST'],
    'user': config['local_mysql']['USER'],
    'password': config['local_mysql']['PASSWORD'],
    'database': config['local_mysql']['DATABASE']
}

postgres_config = {
    'host': config['local_postgres']['HOST'],
    'port': config['local_postgres']['PORT'],
    'user': config['local_postgres']['USER'],
    'password': config['local_postgres']['PASSWORD'],
    'database': config['local_postgres']['DATABASE'],
}

## Conexión base de datos MySQL

In [4]:
mysql_connection = pymysql.connect(**mysql_config)
mysql_cursor = mysql_connection.cursor()

OperationalError: (1049, "Unknown database 'sakila'")

In [25]:
mysql_cursor.execute("SELECT CURDATE()")
mysql_date = mysql_cursor.fetchone()[0]
print(f"Current MySQL date: {mysql_date}")

Current MySQL date: 2025-04-02


### Cerrar conexión a la base de datos [MySQL]

In [21]:
mysql_cursor.close()
mysql_connection.close()

## Conexión base de datos PostgreSQL

In [19]:
postgres_connection = psycopg2.connect(**postgres_config)
postgres_cursor = postgres_connection.cursor()

### Cerrar conexión a la base de datos [PostgreSQL]

In [23]:
postgres_cursor.close()
postgres_connection.close()

## Duckdb

In [34]:
import duckdb

duckdb_connection = duckdb.connect(database="datamart", read_only=False)

duckdb_connection.execute("SELECT * FROM Tbl_Alquiler;")

print(f'Data from DuckDB:{duckdb_connection.fetchall()}')

duckdb_connection.close()

Data from DuckDB:[]
